# Food Bank Problem

In [12]:
import sys
import importlib
import numpy as np
import nbformat
import plotly.express
import plotly.express as px
import pandas as pd
import scipy.optimize as optimization
import food_bank_functions
import food_bank_bayesian
from food_bank_functions import *
from food_bank_bayesian import *
importlib.reload(food_bank_functions)
importlib.reload(food_bank_bayesian)

<module 'food_bank_bayesian' from 'C:\\Users\\seanr\\Documents\\GitHub\\FoodBank\\food_bank_bayesian.py'>

In [22]:
num_iterations = 50
num_towns_range = 100
max_n = 1000
fix_num_towns = 10
max_b = 200
grid_size = .01

In [ ]:
data_dict_3 = {'NumTowns':[],'Dynamic':[],'Dynamic_B+1':[], 'Weighted':[], 'BayesOpt':[], 'BayesOptEst':[]}
   
expected_demands = np.zeros(max_n)+1.5
max_budget = np.sum(expected_demands)

b_grid = np.arange(0, max_budget+grid_size, grid_size)
opt_policy, v_fn = bayes_opt(max_n, max_budget, b_grid, grid_size)
    

for n in np.logspace(0,np.log10(max_n),fix_num_towns):
    n=int(n)
    if n == 0:
        n = 1
    print('Currently on town: ' + str(n))
    
    
    
    avg_1, avg_2, avg_3, avg_4 = 0,0,0,0
    data_dict_3['NumTowns'].append(n)
    for i in range(num_iterations):
        expected_demands = np.zeros(n)+1.5
        budget = np.sum(expected_demands)
        demands = np.random.randint(1, 3, n)
        sorted_distribution = np.arange(2) + 1
        weights = np.zeros(2)+0.5      
        opt = objective_nash_log(demands, waterfilling(demands,budget))
        for j in range(5):
            if j==0:
                avg_1 += (opt - objective_nash_log(demands, waterfilling_dynamic(expected_demands, demands,budget)))
            if j==1:
                avg_2 += (opt - objective_nash_log(demands, waterfilling_weights(weights, sorted_distribution, demands, budget)))
            if j==2:
                avg_3 += (opt - objective_nash_log(demands, waterfilling_dynamic_budget_opt(expected_demands, demands,budget,n/5)))
            if j==3:
                avg_4 += (opt - objective_nash_log(demands, waterfilling_bayesian(demands, opt_policy[(max_n-n):(max_n+1),:], budget, b_grid, grid_size)))
            if j==4:
                avg_5 = (opt-v_fn[max_n-n,len(b_grid)-1])
    data_dict_3['Dynamic'].append(avg_1/num_iterations)
    data_dict_3['Weighted'].append(avg_2/num_iterations)
    data_dict_3['Dynamic_B+1'].append(avg_3/num_iterations)
    data_dict_3['BayesOpt'].append(avg_4/num_iterations)
    data_dict_3['BayesOptEst'].append(avg_5)
    
df_uniform = pd.DataFrame(data_dict_3).melt(id_vars="NumTowns")
fig = px.scatter(df_uniform, x="NumTowns", y="value", color='variable')
fig.update_layout(xaxis_type="log", yaxis_type="log")
fig.show()